In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np
import skimage.io as io

from PIL import Image

import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2

from pathlib import Path

import sklearn.feature_extraction
import sklearn.feature_extraction
from skimage.segmentation import expand_labels, watershed
import skimage.morphology

from sklearn.cluster import spectral_clustering

from skimage import morphology


import deepdrr
from deepdrr import Volume
from deepdrr import geo
from deepdrr.utils import test_utils

import nibabel as nib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
# unique ops

# crop - done
# split loose - done
# dilate - done
# erode - done
# downscale 2x - done
# upscale 2x - done
# spectral cluster - done
# mode blur - done

In [8]:
# vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/pulmonary_vein.nii.gz")
vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/heart.nii.gz")
print(vol_path)
print(vol_path.exists())


/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/heart.nii.gz
True


In [9]:
vol = Volume.from_nifti(vol_path, segmentation=True)
vol

In [10]:
vol.shape

(512, 512, 236)

In [11]:
vol_poly = vol.isosurface(decimation=0, smooth_iter=0, smooth=False,)
vol_poly.save("mesh.stl")
print(vol_poly.bounds)

(-80.154296875, 62.287109375, -179.62200927734375, -68.05950927734375, -486.5, -375.5)


In [12]:
vol_cropped = vol.shrink(padding=5)

In [13]:
vol_cropped.shape

(152, 121, 46)

In [14]:
vol_poly = vol_cropped.isosurface(decimation=0, smooth_iter=0, smooth=False,)
vol_poly.save("cropped_mesh.stl")
print(vol_poly.bounds)


(-80.154296875, 62.287109375, -179.62200927734375, -68.05950927734375, -486.5, -375.5)


In [15]:
bodies = vol_cropped.split_bodies()

In [16]:
vol_data = vol_cropped.data

# vol_data = morphology.binary_erosion(vol_data, footprint=morphology.ball(5))
vol_data = morphology.binary_dilation(vol_data, footprint=morphology.ball(5))

In [17]:
float_labels = vol_data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas2.nii.gz')

In [18]:
body = bodies[0]

body.shape

(152, 121, 46)

In [19]:
body_shrunk = body.shrink(padding=5)
print(body_shrunk.shape)

(152, 121, 46)


In [20]:
shrunk_downscaled = body_shrunk.downscale((3,3,1), segmentation=True)
print(shrunk_downscaled.shape)

float_labels = shrunk_downscaled.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas3.nii.gz')

(51, 41, 46)


In [21]:
segmented = shrunk_downscaled.cluster_seg(segmentation=True, n_clusters=9, expand=10)

float_labels = segmented.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas4.nii.gz')

In [24]:
segmented_upscaled = segmented.upscale((3,3,1), segmentation=True, smooth_seg=(3,3,3))

float_labels = segmented_upscaled.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas5.nii.gz')

In [26]:
print(segmented_upscaled.shape)
print(body_shrunk.shape)

(153, 123, 46)
(152, 121, 46)


In [28]:
split_parts = body_shrunk.split_by_segmentation(segmented_upscaled.data)
print(len(split_parts))

9


In [31]:
# # make an isosurface for each part
# for i, part in enumerate(split_parts):
#     part_poly = part.isosurface(decimation=0, smooth_iter=0, smooth=False,)
#     part_poly.save(f"part_{i}.stl")

# zip parts
import zipfile
with zipfile.ZipFile("parts.zip", "w") as zf:
    for i, part in enumerate(split_parts):
        part_poly = part.isosurface(decimation=0, smooth_iter=0, smooth=False,)
        part_poly.save(f"part_{i}.stl")
        zf.write(f"part_{i}.stl")

In [ ]:
# shrunk_upscaled = shrunk_downscaled.upscale((4,4,2), segmentation=True)
# print(shrunk_upscaled.shape)